In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据路径
train_dir = '/root/datasets/BTXRD3/train'
val_dir = '/root/datasets/BTXRD3/val'
output_dir = '/root/datasets/DenseNet基础模型骨3分类/'

# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 数据预处理和加载
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)
class_names = full_dataset.classes

# 构建DenseNet-121模型并加载2分类任务最佳模型权重
def build_model(num_classes=3):
    model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
    model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    return model

# 加载2分类任务最佳模型权重
best_model_path = '/root/datasets/DenseNet_best.pth'
model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
model.classifier = nn.Linear(model.classifier.in_features, 2)
model.load_state_dict(torch.load(best_model_path, map_location=device))

# 修改模型最后一层以适应3分类任务
model.classifier = nn.Linear(model.classifier.in_features, 3)
model = model.to(device)

# 交叉验证和模型训练
kf = KFold(n_splits=5, shuffle=True, random_state=42)
metrics = []

best_val_loss = float('inf')
best_epoch = -1
best_fold = -1

fold_val_true = []
fold_val_pred = []
fold_val_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(full_dataset)):
    print(f'Fold {fold + 1}')
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
    
    # 构建DenseNet-121模型并加载2分类任务最佳模型权重
    model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
    model.classifier = nn.Linear(model.classifier.in_features, 2)
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    
    # 修改模型最后一层以适应3分类任务
    model.classifier = nn.Linear(model.classifier.in_features, 3)
    model = model.to(device)

    # 冻结除最后一层之外的所有层
    for name, param in model.named_parameters():
        if 'classifier' not in name:
            param.requires_grad = False

    # 重新编译模型
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    
    fold_best_val_loss = float('inf')
    fold_best_epoch = -1
    fold_best_model_path = os.path.join(output_dir, f'best_model_fold_{fold + 1}.pth')

    for epoch in range(20):
        model.train()
        train_loss = 0.0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        model.eval()
        val_loss = 0.0
        val_true = []
        val_pred = []
        val_scores = []
        with torch.no_grad():
            for images, labels in tqdm(val_loader):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                val_true.extend(labels.cpu().numpy())
                val_pred.extend(outputs.argmax(dim=1).cpu().numpy())
                val_scores.extend(torch.softmax(outputs, dim=1).cpu().numpy())
        
        val_loss /= len(val_loader.dataset)
        
        if val_loss < fold_best_val_loss:
            fold_best_val_loss = val_loss
            fold_best_epoch = epoch
            torch.save(model.state_dict(), fold_best_model_path)
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    if fold_best_val_loss < best_val_loss:
        best_val_loss = fold_best_val_loss
        best_epoch = fold_best_epoch
        best_fold = fold + 1
        torch.save(model.state_dict(), os.path.join(output_dir, 'best_downstream_model.pth'))
    
    model.load_state_dict(torch.load(fold_best_model_path))
    val_true = np.array(val_true)
    val_pred = np.array(val_pred)
    val_scores = np.array(val_scores)

    fold_val_true.extend(val_true)
    fold_val_pred.extend(val_pred)
    fold_val_scores.extend(val_scores)
    
    accuracy = accuracy_score(val_true, val_pred)
    precision = precision_score(val_true, val_pred, average='weighted')
    recall = recall_score(val_true, val_pred, average='weighted')
    f1 = f1_score(val_true, val_pred, average='weighted')
    auc = roc_auc_score(val_true, val_scores, multi_class='ovo')
    
    print(f'Best model found at epoch {fold_best_epoch}')
    print(f'Val Loss: {fold_best_val_loss:.4f} Acc: {accuracy:.4f} Precision: {precision:.4f} Recall: {recall:.4f} F1: {f1:.4f} AUC: {auc:.4f}')
    
    class_metrics = {}
    for class_idx, class_name in enumerate(full_dataset.classes):
        class_true = (val_true == class_idx).astype(int)
        class_pred = (val_pred == class_idx).astype(int)
        class_score = val_scores[:, class_idx]
        
        class_accuracy = accuracy_score(class_true, class_pred)
        class_precision = precision_score(class_true, class_pred)
        class_recall = recall_score(class_true, class_pred)
        class_f1 = f1_score(class_true, class_pred)
        class_auc = roc_auc_score(class_true, class_score)
        
        class_metrics[class_name] = {
            'accuracy': class_accuracy,
            'precision': class_precision,
            'recall': class_recall,
            'f1': class_f1,
            'auc': class_auc
        }
        
        print(f'Val Class {class_name} Acc: {class_accuracy:.4f} Precision: {class_precision:.4f} Recall: {class_recall:.4f} F1: {class_f1:.4f} AUC: {class_auc:.4f}')
    
    metrics.append({'fold': fold + 1, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1, 'auc': auc, 'class_metrics': class_metrics, 'best_epoch': fold_best_epoch})

# 保存五折交叉验证的指标
metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv(os.path.join(output_dir, 'cross_validation_metrics_downstream.csv'), index=False)

# 绘制五折交叉验证的ROC曲线
plt.figure()
for class_idx, class_name in enumerate(full_dataset.classes):
    class_true = (np.array(fold_val_true) == class_idx).astype(int)
    class_score = np.array(fold_val_scores)[:, class_idx]
    
    fpr, tpr, _ = roc_curve(class_true, class_score)
    auc = roc_auc_score(class_true, class_score)
    
    plt.plot(fpr, tpr, label=f'{class_name} (area = {auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Cross-Validation ROC Curve')
plt.legend(loc="lower right")
plt.savefig(os.path.join(output_dir, 'cross_validation_roc_curve_downstream.pdf'))
plt.close()

# 绘制五折交叉验证的混淆矩阵
cm = confusion_matrix(fold_val_true, fold_val_pred)
plt.figure()
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Cross-Validation Confusion Matrix')
plt.savefig(os.path.join(output_dir, 'cross_validation_confusion_matrix_downstream.pdf'))
plt.close()

# 加载最终最优模型并评估独立验证集
model.load_state_dict(torch.load(os.path.join(output_dir, 'best_downstream_model.pth')))
model.eval()

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
val_true = []
val_pred = []
val_scores = []
with torch.no_grad():
    for images, labels in tqdm(val_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        val_true.extend(labels.cpu().numpy())
        val_pred.extend(outputs.argmax(dim=1).cpu().numpy())
        val_scores.extend(torch.softmax(outputs, dim=1).cpu().numpy())

val_true = np.array(val_true)
val_pred = np.array(val_pred)
val_scores = np.array(val_scores)

# 计算验证集的指标
accuracy = accuracy_score(val_true, val_pred)
precision = precision_score(val_true, val_pred, average='weighted')
recall = recall_score(val_true, val_pred, average='weighted')
f1 = f1_score(val_true, val_pred, average='weighted')
auc = roc_auc_score(val_true, val_scores, multi_class='ovo')

print(f'Validation Set Metrics:')
print(f'Acc: {accuracy:.4f} Precision: {precision:.4f} Recall: {recall:.4f} F1: {f1:.4f} AUC: {auc:.4f}')

# 保存独立验证集的指标
val_metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1,
    'auc': auc,
    'class_metrics': {}
}

for class_idx, class_name in enumerate(val_dataset.classes):
    class_true = (val_true == class_idx).astype(int)
    class_pred = (val_pred == class_idx).astype(int)
    class_score = val_scores[:, class_idx]
    
    class_accuracy = accuracy_score(class_true, class_pred)
    class_precision = precision_score(class_true, class_pred)
    class_recall = recall_score(class_true, class_pred)
    class_f1 = f1_score(class_true, class_pred)
    class_auc = roc_auc_score(class_true, class_score)
    
    val_metrics['class_metrics'][class_name] = {
        'accuracy': class_accuracy,
        'precision': class_precision,
        'recall': class_recall,
        'f1': class_f1,
        'auc': class_auc
    }
    
    print(f'Val Class {class_name} Acc: {class_accuracy:.4f} Precision: {class_precision:.4f} Recall: {class_recall:.4f} F1: {class_f1:.4f} AUC: {class_auc:.4f}')

# 保存独立验证集的指标
with open(os.path.join(output_dir, 'validation_metrics_downstream.csv'), 'w') as f:
    for key in val_metrics.keys():
        f.write(f"{key},{val_metrics[key]}\n")

# 绘制独立验证集的ROC曲线
plt.figure()
for class_idx, class_name in enumerate(val_dataset.classes):
    class_true = (val_true == class_idx).astype(int)
    class_score = val_scores[:, class_idx]
    
    fpr, tpr, _ = roc_curve(class_true, class_score)
    auc = roc_auc_score(class_true, class_score)
    
    plt.plot(fpr, tpr, label=f'{class_name} (area = {auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Validation ROC Curve')
plt.legend(loc="lower right")
plt.savefig(os.path.join(output_dir, 'validation_roc_curve_downstream.pdf'))
plt.close()

# 绘制独立验证集的混淆矩阵
cm = confusion_matrix(val_true, val_pred)
plt.figure()
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Validation Confusion Matrix')
plt.savefig(os.path.join(output_dir, 'validation_confusion_matrix_downstream.pdf'))
plt.close()

# 输出最终最优模型的折数和最佳轮次
print(f'The best model was found in fold {best_fold} at epoch {best_epoch}')


Fold 1


100%|██████████| 21/21 [00:18<00:00,  1.11it/s]


Epoch 1, Train Loss: 0.9491, Val Loss: 0.8410


100%|██████████| 21/21 [00:18<00:00,  1.15it/s]


Epoch 2, Train Loss: 0.7897, Val Loss: 0.7816


...

Epoch 19, Train Loss: 0.5546, Val Loss: 0.5854


100%|██████████| 21/21 [00:18<00:00,  1.12it/s]


Epoch 20, Train Loss: 0.5451, Val Loss: 0.5890
Best model found at epoch 18
Val Loss: 0.5854 Acc: 0.7538 Precision: 0.7543 Recall: 0.7538 F1: 0.7537 AUC: 0.8973
Val Class benign Acc: 0.7948 Precision: 0.6597 Recall: 0.6429 F1: 0.6512 AUC: 0.8364
Val Class malignant Acc: 0.8982 Precision: 0.8457 Recall: 0.8071 F1: 0.8260 AUC: 0.9523
Val Class normal Acc: 0.8146 Precision: 0.7563 Recall: 0.7962 F1: 0.7757 AUC: 0.9009


100%|██████████| 35/35 [00:35<00:00,  1.02s/it]


Validation Set Metrics:
Acc: 0.7116 Precision: 0.7103 Recall: 0.7116 F1: 0.7103 AUC: 0.8383
Val Class benign Acc: 0.7432 Precision: 0.6904 Recall: 0.6401 F1: 0.6643 AUC: 0.8050
Val Class malignant Acc: 0.9159 Precision: 0.5472 Recall: 0.5631 F1: 0.5550 AUC: 0.8692
Val Class normal Acc: 0.7640 Precision: 0.7555 Recall: 0.7943 F1: 0.7744 AUC: 0.8370
The best model was found in fold 5 at epoch 18
